In [1]:
%load_ext autoreload
%autoreload 2

In [35]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from meteostat import Stations, Daily, Hourly, units

from hourly import *
from daily import *
from stations import *

In [36]:
stations = get_stations()

In [37]:
station_id = ['KIAG0']

hourly_weather = get_hourly_weather(station = station_id, 
                   days_history = 60)

In [38]:
daily_prec = (hourly_weather
    .groupby('time_date')
    .agg({'prcp': 'sum'})
    .pipe(lambda x: x.assign(total_prcp_7_days = x['prcp'].rolling(7).sum()))
    .pipe(lambda x: x.assign(total_prcp_30_days = x['prcp'].rolling(30).sum()))
    .reset_index()
    .sort_values(['time_date'], ascending = True)
    .tail(30)
    .rename(columns = {'prcp': "Total_Daily",
                      'total_prcp_7_days': "7_Day_Total",
                      'total_prcp_30_days': '30_Day_Total'})
    .pipe(lambda x: pd.melt(x, id_vars = ['time_date'], value_vars = ['Total_Daily', '7_Day_Total', '30_Day_Total']))
)

In [44]:
import streamlit as st
help(st.vega_lite_chart)

Help on method vega_lite_chart in module streamlit.delta_generator:

vega_lite_chart(data=None, spec=None, use_container_width=False, **kwargs) method of streamlit.delta_generator.DeltaGenerator instance
    Display a chart using the Vega-Lite library.
    
    Parameters
    ----------
    data : pandas.DataFrame, pandas.Styler, numpy.ndarray, Iterable, dict,
        or None
        Either the data to be plotted or a Vega-Lite spec containing the
        data (which more closely follows the Vega-Lite API).
    
    spec : dict or None
        The Vega-Lite spec for the chart. If the spec was already passed in
        the previous argument, this must be set to None. See
        https://vega.github.io/vega-lite/docs/ for more info.
    
    use_container_width : bool
        If True, set the chart width to the column width. This takes
        precedence over Vega-Lite's native `width` value.
    
    **kwargs : any
        Same as spec, but as keywords.
    
    Example
    -------
    

In [23]:
daily_prec

,time_date,variable,value
0,2021-07-19,prcp,0.024
1,2021-07-18,prcp,0.008
2,2021-07-17,prcp,3.290
3,2021-07-16,prcp,0.216
4,2021-07-15,prcp,0.000
...,...,...,...
85,2021-06-24,total_prcp_30_days,2.990
86,2021-06-23,total_prcp_30_days,2.990
87,2021-06-22,total_prcp_30_days,2.986
88,2021-06-21,total_prcp_30_days,2.986
